In [9]:
import os
import pandas as pd
from collections import defaultdict

In [10]:
# Ruta a los archivos (desde ETL)
folder_path = "../DataLake"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Guardar columnas por dataset
columnas_por_dataset = {}
columnas_totales = set()

# Paso 1: recolectar columnas
for file in csv_files:
    dataset_name = file.replace(".csv", "")
    path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(path, nrows=1)  # Solo una fila para cargar las columnas
        columnas = list(df.columns)
        columnas_por_dataset[dataset_name] = columnas
        columnas_totales.update(columnas)
    except Exception as e:
        print(f"❌ Error en {dataset_name}: {e}")

# Paso 2: contar en cuántos datasets aparece cada columna
columna_frecuencia = defaultdict(int)

for columnas in columnas_por_dataset.values():
    for col in columnas:
        columna_frecuencia[col] += 1

# Convertir a DataFrame para visualizar
frecuencia_df = pd.DataFrame.from_dict(columna_frecuencia, orient="index", columns=["# datasets"])
frecuencia_df = frecuencia_df.sort_values(by="# datasets", ascending=False)

# Paso 3: columnas presentes en TODOS los datasets
num_datasets = len(columnas_por_dataset)
columnas_comunes = frecuencia_df[frecuencia_df["# datasets"] == num_datasets]

# Paso 4: columnas faltantes por dataset
faltantes_por_dataset = {}

for dataset, columnas in columnas_por_dataset.items():
    faltantes = columnas_totales - set(columnas)
    faltantes_por_dataset[dataset] = faltantes

# Mostrar resultados
print(f"🔍 Columnas totales distintas: {len(columnas_totales)}")
print(f"✅ Columnas comunes a TODOS los datasets ({len(columnas_comunes)}):")
print(columnas_comunes.index.tolist())

print("\n📉 Top columnas más frecuentes:")
display(frecuencia_df.head(10))

print("\n📂 Ejemplo de columnas faltantes en un dataset:")
for dataset, faltantes in list(faltantes_por_dataset.items())[:1]:
    print(f"\n🗂 {dataset} (faltan {len(faltantes)} columnas):")
    print(sorted(faltantes))


🔍 Columnas totales distintas: 104
✅ Columnas comunes a TODOS los datasets (59):
['sample_id', 'name.project', 'site_of_resection_or_biopsy.diagnoses', 'sample_type_id.samples', 'sample_type.samples', 'sample_id.samples', 'age_at_diagnosis.diagnoses', 'race.demographic', 'project_id.project', 'project.tissue_source_site', 'progression_or_recurrence.diagnoses', 'prior_treatment.diagnoses', 'prior_malignancy.diagnoses', 'primary_site.project', 'primary_site', 'primary_diagnosis.diagnoses', 'preservation_method.samples', 'specimen_type.samples', 'state.treatments.diagnoses', 'submitter_id', 'tumor_descriptor.samples', 'year_of_diagnosis.diagnoses', 'year_of_death.demographic', 'year_of_birth.demographic', 'vital_status.demographic', 'updated_datetime.treatments.diagnoses', 'tumor_grade.diagnoses', 'treatment_type.treatments.diagnoses', 'submitter_id.treatments.diagnoses', 'treatment_or_therapy.treatments.diagnoses', 'treatment_id.treatments.diagnoses', 'tissue_type.samples', 'tissue_source

,# datasets
sample_id,33
name.project,33
site_of_resection_or_biopsy.diagnoses,33
sample_type_id.samples,33
sample_type.samples,33
sample_id.samples,33
age_at_diagnosis.diagnoses,33
race.demographic,33
project_id.project,33
project.tissue_source_site,33



📂 Ejemplo de columnas faltantes en un dataset:

🗂 TCGA-THYM.clinical (faltan 25 columnas):
['ajcc_clinical_m.diagnoses', 'ajcc_clinical_n.diagnoses', 'ajcc_clinical_stage.diagnoses', 'ajcc_clinical_t.diagnoses', 'ajcc_pathologic_m.diagnoses', 'ajcc_pathologic_n.diagnoses', 'ajcc_pathologic_stage.diagnoses', 'ajcc_pathologic_t.diagnoses', 'ajcc_staging_system_edition.diagnoses', 'ann_arbor_b_symptoms.diagnoses', 'ann_arbor_clinical_stage.diagnoses', 'ann_arbor_extranodal_involvement.diagnoses', 'annotations.samples', 'cigarettes_per_day.exposures', 'composition.samples', 'days_to_sample_procurement.samples', 'figo_stage.diagnoses', 'igcccg_stage.diagnoses', 'intermediate_dimension.samples', 'longest_dimension.samples', 'pack_years_smoked.exposures', 'primary_gleason_grade.diagnoses', 'secondary_gleason_grade.diagnoses', 'shortest_dimension.samples', 'years_smoked.exposures']


In [12]:
import os
import pandas as pd

# Ruta a DataLake (desde ETL)
datalake_path = "../DataLake"
csv_files = [f for f in os.listdir(datalake_path) if f.endswith(".csv")]

# Crear diccionario para mapear columnas presentes
columnas_por_dataset = {}
todas_las_columnas = set()

for file in csv_files:
    dataset_name = file.replace(".csv", "")
    path = os.path.join(datalake_path, file)
    
    try:
        df = pd.read_csv(path, nrows=1)
        columnas = list(df.columns)
        columnas_por_dataset[dataset_name] = columnas
        todas_las_columnas.update(columnas)
    except Exception as e:
        print(f"❌ Error en {dataset_name}: {e}")

# Crear DataFrame booleano
todas_las_columnas = sorted(todas_las_columnas)
boolean_df = pd.DataFrame(index=columnas_por_dataset.keys(), columns=todas_las_columnas)

for dataset, columnas in columnas_por_dataset.items():
    boolean_df.loc[dataset] = [1 if col in columnas else 0 for col in todas_las_columnas]

# Asegurar que los valores son enteros
boolean_df = boolean_df.astype(int)

# Guardar en la carpeta Utilidades
output_path = "../Utilidades/tabla_columnas_booleanas.csv"
os.makedirs("../Utilidades", exist_ok=True)
boolean_df.to_csv(output_path)

print(f"✅ Tabla guardada en: {output_path}")


✅ Tabla guardada en: ../Utilidades/tabla_columnas_booleanas.csv


In [16]:
pip install qgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 889.2/889.2 kB 6.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for qgrid: filename=qgrid-1.3.1-py2.py3-none-any.whl size=1761255 sha256=3cca8374ccb962f0cfa7be476034eef64fa17fb7b093b92e72294df079a1a19d
  Stored in directory: /Users/carlotasanchezgonzalez/Library/Caches/pip/wheels/7f/0f/d6/33b60aa35dbdc3d3e84c44b780c85e92d8767b698843f9e256
Successfully built qgrid
Note: you may need to restart the kernel to use updated packages.
